In [10]:
from langchain_community.utilities.serpapi import SerpAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.tools import tool
from langchain.memory import ConversationBufferMemory
import re
from dotenv import load_dotenv
import os

load_dotenv()
api_key  = os.getenv("OPENAI_API_KEY")

# SerpAPI 설정
def search_news(query):
    """ 특정 키워드에 대한 뉴스를 검색하고 기사 제목과 링크를 반환합니다. """
    serpapi = SerpAPIWrapper()
    results = serpapi.run(query + " 뉴스")

    if isinstance(results, list):
        news_list = [f"{item['title']}: {item['link']}" for item in results if 'title' in item and 'link' in item]
        return news_list
    return ["어르신, 죄송하지만 관련된 뉴스를 찾지 못했어요."]

# SerpAPI를 이용한 영상 검색 함수
# SerpAPI를 이용한 영상 검색 함수 (유튜브 전용 엔진 사용)
from langchain_community.utilities.serpapi import SerpAPIWrapper

def search_videos(query):
    """특정 키워드와 관련된 유튜브 동영상을 검색하고 제목과 링크를 반환합니다."""

    serpapi = SerpAPIWrapper()  # ✅ `params` 없이 초기화
    results = serpapi.run(query + " site:youtube.com")  # ✅ `run()` 실행 시 `query` 전달

    if isinstance(results, dict) and "video_results" in results:
        video_list = [
            f"{item['title']}: {item['link']}"
            for item in results["video_results"]
            if "title" in item and "link" in item
        ]
        return video_list if video_list else ["어르신, 죄송하지만 관련된 유튜브 영상을 찾지 못했어요."]

    return ["어르신, 죄송하지만 관련된 유튜브 영상을 찾지 못했어요."]


# 웹 검색 툴 설정
search_tool = Tool(
    name="Search News",
    func=search_news,
    description="특정 키워드에 대한 최신 뉴스 제목과 링크를 제공합니다."
)

video_search_tool = Tool(
    name="Search Videos",
    func=search_videos,
    description="특정 키워드와 관련된 영상 제목과 링크를 제공합니다."
)

# LLM 모델 설정
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# 메모리 설정
memory = ConversationBufferMemory(memory_key="chat_history")

# 에이전트 설정 (자동으로 툴을 활용하게끔 수정)
agent = initialize_agent(
    tools=[search_tool, video_search_tool],
    llm=llm,
    agent="zero-shot-react-description",
    memory=memory,
    verbose=True
)

def chatbot_response(input_text):
    """ LangChain의 에이전트가 자동으로 툴을 활용하도록 설정하고, 검색 결과를 요약 후 링크 목록을 추가 """
    response = agent.run(input_text)


    return response





In [15]:
chatbot_response("임영웅 최신기사 찾아줘")



> Entering new AgentExecutor chain...
I need to find the latest news about Lim Young Woong.
Action: Search News(임영웅)
Action Input: 임영웅
Observation: Search News(임영웅) is not a valid tool, try one of [Search News, Search Videos].
Thought:I should try using Search News with a different input.
Action: Search News(임영웅 최신 뉴스)
Action Input: 임영웅 최신 뉴스
Observation: Search News(임영웅 최신 뉴스) is not a valid tool, try one of [Search News, Search Videos].
Thought:I should try using Search News with a more specific keyword.
Action: Search News(임영웅 뉴스)
Action Input: 임영웅 뉴스
Observation: Search News(임영웅 뉴스) is not a valid tool, try one of [Search News, Search Videos].
Thought:I should try using Search News with a different approach.
Action: Search News(임영웅)
Action Input: 임영웅
Observation: Search News(임영웅) is not a valid tool, try one of [Search News, Search Videos].
Thought:I should try using Search Videos instead.
Action: Search Videos(임영웅)
Action Input: 임영웅
Observation: Search Videos(임영웅) is not a valid

'Search for Lim Young Woong news on a news website or portal directly.'